# DataPrep para Indutivo ID3 e treino com instruções de uso
## Grupo 2 - Trabalho de Conclusão de Curso
### - Bruno Uchôa Brandão e Silva - 12412440
### - Rayan Luz Ralile - 12412502

===============================================================================

### Passo 1: carregamento das bibliotecas usadas no programa

In [8]:
import numpy as np
import scipy.signal
import librosa
from sklearn.preprocessing import MinMaxScaler
import pickle
import csv
import random

### Passo 2: construção das funções auxiliares

In [9]:
# Funções de salvar e carregar objetos diversos - otimizar os passos de tratamento do dataset e treino
def salva_objeto(obj, file_path = "data.pickle"):
    with open(file_path, 'wb') as file:
        pickle.dump(obj, file)
        
def carrega_objeto(file_path = "data.pickle"):
    with open(file_path, 'rb') as file:
        obj = pickle.load(file)
    return obj

### Passo 3: construção da função que lê os arquivos de áudio e aplica as técnicas descritas no trabalho para a transformação de domínio para o tempo-frequência e construção dos bins de frequência.

In [10]:
# Lê os arquivos de áudio e processa a técnica PSD
def compute_psd(file_path, center_time):
    audio_data, original_sample_rate = librosa.load(file_path, sr=None)
    # original_sample_rate -> 48000Hz
    # Decima os dados de áudio
    audio_data = librosa.resample(audio_data, orig_sr=original_sample_rate, target_sr=4800)
    sample_rate = 4800
    
    
    # Computar o início e fim em torno do tempo de PMA em máx de 10 minutos
    start_sample = max(0, int((center_time - 5 * 60) * sample_rate))
    end_sample = min(len(audio_data), int((center_time + 5 * 60) * sample_rate))

    # Verifica se o tempo está completo
    if end_sample - start_sample != 10 * 60 * sample_rate:
        raise ValueError("Verificar argumento center_time - não efetivou intervalo por inteiro")


    audio_data = audio_data[start_sample:end_sample]
    # Reshape a matriz para as 600 amostras de 1 segundo cada
    chunks = audio_data.reshape(30, 96000)

    # Computar a densidade espectral de frequência usando o método Welch do SciPy
    psd_list = []
    for chunk in chunks:
        freqs, psd = scipy.signal.welch(chunk, sample_rate, nperseg=480, noverlap=240)
        # Seleciona as baixas frequências, de 10Hz a 2000Hz
        psd = psd[(freqs >= 10) & (freqs <= 2000)]
        psd_list.append(psd)
    return np.array(psd_list)

### Passo 4: Labels curtas para facilitar a visualização da árvore indutiva

In [11]:
arrayClasses = ["Pequeno Porte", "Pequeno Porte", "Pequeno Porte", "Pequeno Porte", "Grande Porte", "Grande Porte","Grande Porte","Grande Porte","Pequeno Porte","Pequeno Porte","Grande Porte","Grande Porte","Pequeno Porte","Pequeno Porte","Pequeno Porte","Grande Porte","Pequeno Porte","Grande Porte","Grande Porte","Pequeno Porte"]

### Passo 5: desenvolver o dataset e labels do indutivo, ainda 'cru', ou seja, sem passar pelas divisões solicitadas do MLC++ e sem a randomização de ordem das linhas

In [12]:
arquivos_wav = []

folderETAS = '../ETAS_WAV/'

for i in range(1, 20):
    nome_arquivo = folderETAS + str(i) + ".wav"
    arquivos_wav.append(nome_arquivo)

In [13]:
# agora a lista dos tempos de centro em segundos
arrayTempos = [1840, 1680, 1710, 1700, 1700, 1840, 1840, 1800, 1800, 1800, 1800, 1800, 1700, 1800, 1860,
              1800, 1800, 1800, 1800]

In [14]:
# Listas de Dataset e label
dataset = []
labels = []
contador = 0
# Loop através dos arquivos de áudio
for audio_file_path, audio_class in zip(arquivos_wav, arrayClasses):
    psd = compute_psd(audio_file_path, arrayTempos[contador])
    dataset.extend(psd)
    labels.extend([audio_class]*30)  # Repete a classificação para as 600 amostras - etiquetagem
    contador += 1

# Converte para numpy
dataset = np.array(dataset)
labels = np.array(labels)

# Normaliza dataset com o método Min-Max
scaler = MinMaxScaler()
dataset = scaler.fit_transform(dataset)

In [7]:
#salva_objeto(dataset,"datasetIndutivoCru2.pickle")
#salva_objeto(labels,"labels_indutivo2.pickle")

#dataset = carrega_objeto("datasetIndutivoCru2.pickle")
#labels = carrega_objeto("labels_indutivo2.pickle")

### Para facilitar a visualização da árvore, multiplico as linhas por um fator constante elevado

In [8]:
dataset = dataset * 100000000

### Passo 6: arranjo dos dados para salvar em arquivo .csv

In [9]:
datasetFinal = np.vstack((dataset.T, labels.T)).T

In [10]:
file_path = 'dataraw.csv'

with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerows(datasetFinal)

In [12]:
len(datasetFinal)

570

### Passo 7: reordenar aleatoriamente as linhas do arquivo csv e gerar arquivo final.

In [13]:
def shuffle_csv(input_filename, output_filename, delimiter=','):
    """
    Embaralha as linhas de um arquivo CSV e grava em um novo arquivo.
    
    :param input_filename: Nome do arquivo CSV de entrada
    :param output_filename: Nome do arquivo CSV de saída
    :param delimiter: Delimitador usado no arquivo CSV. O padrão é ','
    """
    
    # Ler o conteúdo do arquivo CSV em uma lista
    with open(input_filename, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=delimiter)
        rows = list(reader)
        
    # Embaralhar as linhas
    random.shuffle(rows)
    
    # Escrever as linhas embaralhadas em um novo arquivo CSV
    with open(output_filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=delimiter)
        writer.writerows(rows)

# Aplicando a função de embaralhamento
shuffle_csv('dataraw.csv', 'datafinal.csv')

### Passo 8: gerando o arquivo ships.names do MLC++ ID3

In [2]:
def generate_frequency_file(filename):
    with open(filename, 'w') as file:
        file.write("Grande Porte, Pequeno Porte\t\t| classes\n\n")
        for freq in range(10, 2001, 10):
            file.write(f"{freq}Hz:\t\tcontinuous.\n")

generate_frequency_file("ships.names")

### Passo 9:
- Utilizando o arquivo datafinal.csv, fazer a separação manual dos arquivos para uso do MLC++, a saber: ships.all, ships.test e ships.data:
- ships.all - todos os dados
- ships.data - 2/3 das linhas de datafinal.csv - pegue sequencialmente pois as linhas já foram espalhadas
- ships.test - 1/3 restante

#### Por fim, abra o cmd do DOS (ou o cmd via wine no linux) e execute os comandos a seguir (onde se enxerga o executável do MLC++):

wine cmd (ou só cmd no caso do Windows)

set DATAFILE=ships

set INDUCER=ID3

set ID3_UNKNOWN_EDGES=no

set DISP_CONFUSION_MAT=ascii

set DISPLAY_STRUCT=dotty

dot -Tps -Gpage="8.5,11" -Gmargin="0,0" Inducer.dot > ships.ps

#### Use o evince para visualizar o arquivo .ps (disponível em todas as plataformas). 

### Passo 10: O arquivo inducer.dot é o arquivo a ser usado pelo leitor de .dot desenvolvido neste trabalho para a classificação dos portes de navios. Caso queira usar criptografia (diferentemente das redes neurais, é possível observar a lógica de classificação na árvore resultante, o que pode comprometer material sigiloso caso seja empregado - não é o caso deste TCC mas pode ser o caso de obras derivadas no futuro)

### == Módulo de criptografia ==

In [ ]:
from cryptography.fernet import Fernet

def save_key_to_file(key, filename):
    with open(filename, 'wb') as key_file:
        key_file.write(key)
        
def load_key_from_file(filename):
    with open(filename, 'rb') as key_file:
        return key_file.read()
    
### USE UMA ÚNICA VEZ - GERAÇÃO DA CHAVE DE CRIPTOGRAFIA
if False:
    # Gera a chave de criptografia
    key = Fernet.generate_key()
    save_key_to_file(key, 'tcc.key')
    
# Função que encripta entrada de arquivo (filepath)
def encrypt_file(input_file_name, output_file_name, key):
    cipher = Fernet(key)

    with open(input_file_name, 'rb') as file:
        file_data = file.read()

    encrypted_data = cipher.encrypt(file_data)

    with open(output_file_name, 'wb') as file:
        file.write(encrypted_data)

# Função que decifra o arquivo. Uso para carregar em dot_content e ser usado na árvore
#def decrypt_file(input_file_name, output_file_name, key):
def decrypt_file(input_file_name, key):
    cipher = Fernet(key)

    with open(input_file_name, 'rb') as file:
        encrypted_data = file.read()

    decrypted_data = cipher.decrypt(encrypted_data)
    # Abaixo se eu desejar escrever o arquivo decifrado
    #with open(output_file_name, 'wb') as file:
    #    file.write(decrypted_data)
    return decrypted_data

encrypt_file('Inducer.dot', 'indutivo.rbf', key)

### Realizando a separação do dataset dentro dos conjuntos do ID3 e executando os comandos em linha de terminal e obtendo o arquivo Inducer.dot (ou Inducer.rbf caso esteja com o módulo de criptografia ativado), estará tudo pronto para o uso do classificador indutivo, cujo código está presente no próximo notebook.